In [25]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LSTM, TimeDistributed, Flatten
# from tensorflow.keras.models import Model

# # Input shape for frames (e.g., 224x224, 3 channels for RGB)
# frame_shape = (224, 224, 3)
# sequence_length = 30  # Number of frames to consider in the sequence (tunable)

# # Define CNN model for feature extraction (ResNet50)
# cnn_base = ResNet50(weights='imagenet', include_top=False, input_shape=frame_shape)
# cnn_out = GlobalAveragePooling2D()(cnn_base.output)
# cnn_model = Model(inputs=cnn_base.input, outputs=cnn_out)

# # Input for a sequence of frames
# video_input = Input(shape=(sequence_length, 224, 224, 3))

# # Apply CNN on each frame using TimeDistributed
# cnn_features = TimeDistributed(cnn_model)(video_input)

# # Apply LSTM to capture temporal patterns across frames
# lstm_out = LSTM(128)(cnn_features)

# # Output layer for binary classification (fake or real)
# output = Dense(1, activation='sigmoid')(lstm_out)

# # Define final model
# model = Model(inputs=video_input, outputs=output)

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # Model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 30, 224, 224,   │             0 │
│                                 │ 3)                     │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 30, 2048)       │    23,587,712 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │     1,114,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,702,465 (94.23 MB)

 Trainable params: 24,649,345 (94.03 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [1]:
def extract_frames_from_video(video_path, num_frames=30, frame_size=(224, 224)):
    """
    Extracts a fixed number of frames from a video file.

    Args:
        video_path (str): Path to the video file.
        num_frames (int): Number of frames to extract.
        frame_size (tuple): Desired size of the extracted frames.

    Returns:
        frames_list (list): List of extracted frames as numpy arrays.
    """
    frames_list = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, frame_size)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames_list.append(frame)
        else:
            # If frame extraction fails, append a zero array
            frames_list.append(np.zeros((*frame_size, 3), dtype=np.uint8))
    cap.release()
    return np.array(frames_list)

video_path = 'sample1.mp4'
frames = extract_frames_from_video(video_path)
print(frames.shape)  # Output: (30, 224, 224, 3)



NameError: name 'cv2' is not defined

In [2]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_rate=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    interval = int(fps / frame_rate)

    for i in range(0, frame_count, interval):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_folder, f'frame_{i}.jpg')
            cv2.imwrite(frame_filename, frame)
    
    cap.release()

# Example usage:
extract_frames('sample.mp4', 'output_frames', frame_rate=1)


In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LSTM, TimeDistributed, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

# Input dimensions should match the processed frame size
input_shape = (None, 224, 224, 3)

# Feature extraction using ResNet50
cnn_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Apply TimeDistributed to handle sequence of frames
cnn_out = TimeDistributed(cnn_base)(Input(shape=input_shape))
cnn_out = TimeDistributed(GlobalAveragePooling2D())(cnn_out)

# LSTM for temporal analysis
lstm_out = LSTM(128, return_sequences=False)(cnn_out)

# Classification
output = Dense(1, activation='sigmoid')(lstm_out)

model = Model(inputs=cnn_base.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, None, 224, │          0 │ -                 │
│ (InputLayer)        │ 224, 3)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, None, 7,   │ 23,587,712 │ input_layer_1[0]… │
│ (TimeDistributed)   │ 7, 2048)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, None,      │          0 │ time_distributed… │
│ (TimeDistributed)   │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │  1,114,624 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │        129 │ lstm[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,702,465 (94.23 MB)

 Trainable params: 24,649,345 (94.03 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define the data generators with validation split
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Training data generator (80% of data)
train_generator = train_datagen.flow_from_directory(
    'FF++',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Set to training subset
)

# Validation data generator (20% of data)
validation_generator = train_datagen.flow_from_directory(
    'FF++',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Set to validation subset
)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callbacks for saving the best model and early stopping
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    callbacks=[checkpoint, early_stopping],
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)



Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


ValueError: Must provide at least one structure